In [37]:
from haystack.nodes import FARMReader, BM25Retriever
from haystack.document_stores import InMemoryDocumentStore
from haystack.pipelines.standard_pipelines import TextIndexingPipeline
from GPUtil import showUtilization as gpu_usage
from haystack.pipelines import ExtractiveQAPipeline
from numba import cuda
import logging
import torch
import os

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [38]:
BASE_DOCUMENTS_DIR = "gi_datasets"
BASE_SQAC_DIR = "gi_sqac_ds"
SQAC_AW = "gi_answers.json"
FT_MODEL = "hy_fine_trained"
BASE_MODEL_FOR_FT = "IIC/roberta-base-spanish-sqac" #"distilbert-base-uncased-distilled-squad"


In [39]:
def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

In [40]:
#free_gpu_cache()

In [41]:
ft_reader = FARMReader(model_name_or_path=BASE_MODEL_FOR_FT, use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'IIC/roberta-base-spanish-sqac' (Roberta)
INFO - haystack.modeling.model.language_model -  Auto-detected model language: spanish
INFO - haystack.modeling.model.language_model -  Loaded 'IIC/roberta-base-spanish-sqac' (Roberta model) from model hub.
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


In [42]:

ft_reader.train(data_dir=BASE_SQAC_DIR, train_filename=SQAC_AW, use_gpu=True, n_epochs=20, save_dir=FT_MODEL)


INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.data_handler.data_silo -  
Loading data into the data silo ... 
              ______
               |o  |   !
   __          |:`_|---'-.
  |__|______.-/ _ \-----.|
 (o)(o)------'\ _ /     ( )
 
INFO - haystack.modeling.data_handler.data_silo -  LOADING TRAIN DATA
INFO - haystack.modeling.data_handler.data_silo -  ==================
INFO - haystack.modeling.data_handler.data_silo -  Loading train set from: gi_sqac_ds/gi_answers.json 
Preprocessing dataset:   0%|          | 0/1 [00:00<?, ? Dicts/s]WARNING - haystack.modeling.data_handler.processor -  Answer using start/end indices is 'mbre real es Ei, es ' while gold label text is 'su nombre real es Ei'.
Example will not be converted for training/evaluation.
WARNING - haystack.modeling.data_handler.processor -  Answer using start/end indices is ' encuentra meditando dentro del Plano de' while gold label text is 'meditando dentro del Plano

In [43]:
document_store = InMemoryDocumentStore(use_bm25=True)
retriever = BM25Retriever(document_store=document_store)

INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1


In [44]:

files_to_index = [BASE_DOCUMENTS_DIR + "/" + f for f in os.listdir(BASE_DOCUMENTS_DIR)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)


INFO - haystack.pipelines.base -  It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Updating BM25 representation...: 100%|██████████| 13/13 [00:00<00:00, 19827.62 docs/s]


{'documents': [<Document: {'content': '==Shogun Raiden==\n\n==Info==\nLa Shogun Raiden (en japonés: 雷電 将軍 Raiden Shougun), cuyo nombre real es Ei, es un personaje jugable en Genshin Impact.\nElla controla una marioneta mientras medita dentro del Plano de la eutimia. Ella es el recipiente mortal de Beelzebul, la actual Arconte Electro de Inazuma.\nHizo su primera aparición como PNJ en la Versión 2.0 y llegó por primera vez en la Versión 2.1 en el gachapón "Reino de la serenidad".\n\n==Perfil==\nGobernante indiscutible de toda Inazuma\nLlevando el nombre de "Su Excelencia, la todopoderosa Narukami", le prometió a los habitantes de Inazuma la inmutable eternidad.\nHa destruido el amor con el destello de un relámpago, forjando una tierra pura a partir de un corazón solitario.\nDurante estos largos años, el camino hacia la eternidad ha sido oscuro y lejano, pero Su Alteza Shogun no ha dudado ni un momento.\nSolo en el silencio, la eternidad inmutable revela su naturaleza serena.\n\n==Aparie

In [45]:
ftd_reader = FARMReader(model_name_or_path=FT_MODEL, use_gpu=True)
#ftd_reader = FARMReader(model_name_or_path=BASE_MODEL_FOR_FT, use_gpu=True)


INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.utils -  Using devices: CUDA:0 - Number of GPUs: 1
INFO - haystack.modeling.model.language_model -   * LOADING MODEL: 'hy_fine_trained' (Roberta)
INFO - haystack.modeling.model.language_model -  Loaded 'hy_fine_trained' (Roberta model) from local file system.
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
INFO - haystack.modeling.model.prediction_head -  Loading prediction head from hy_fine_trained/prediction_head_0.bin
INFO - haystack.model

In [46]:
pipe = ExtractiveQAPipeline(ftd_reader, retriever)


In [47]:
queries = [
    "¿De quién es el recipiente Ei?",
    "¿Cuál es el nombre de la Shogun Raiden en japonés?",
    "¿Cuál es la habilidad básica de la Shogun Raiden?",
    "¿Qué estudia Laila?",
    "¿Cuál es el aspecto de Laila?"
]

responses = []

for q in queries:
    prediction = pipe.run(query=q, params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}})
    responses.append(prediction["answers"][0].answer)

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  6.42 Batches/s]


In [48]:
responses


['el Plano de la Eutimia',
 '雷電 将軍 Raiden Shougun',
 'Habilidad elemental',
 'Estudiante de la Facultad Rtawahist, Laila se está especializando en astrología teórica y dibuja incansablemente mapas astrales que incluir en su tesis.',
 'Laila es una joven delgada de piel clara y ojos dorados con pupilas blancas. Tiene el pelo azul oscuro, que se va aclarando en las puntas, y está peinado con coletas que giran en las puntas y orejas puntiagudas. Su rostro está enmarcado por un flequillo que también gira en las puntas y que llega hasta más allá de la barbilla y hasta los hombros. Lleva un tocado dorado que parece una tiara adornada con un adorno de copos de nieve y una capucha puntiaguda de color índigo.']